In [1]:

import torch
from binNF.normalizing_flows.manager import *
import matplotlib.pyplot as plt
import numpy as np

In [66]:
#def g(x):
#    return torch.exp(-torch.sum(x**2,axis=-1)) #Vanilla Gaussian
"""
def g(x): #ring, expect 0.51 in 2 dim
    q=torch.sqrt(torch.sum(x**2,axis=-1))
    #print(q)
    f=q.clone()
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
    z=torch.where(q>0.45,a,q)
    #print(z)
    #print(torch.where(z>1,a,b))
    return torch.where(z<0.2,a,b)
"""
def g(x): #box: expect 16 in dim 2, 64 in dim 2 
   
    q=torch.max(torch.abs(x),dim=-1).values
  
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
   
    return torch.where(q>2,a,b)

def tanp(x):
    return (1+((torch.tan((x-0.5)*np.pi))**2))*np.pi   #derivative for jacobian

n_flow=3
w = torch.empty((200000,n_flow)) 
torch.nn.init.uniform_(w)
v=torch.tan((w-0.5)*(np.pi))

print("Value of the integral before mapping by CL")
print(torch.mean(torch.mul(g(v),torch.prod(tanp(w),axis=-1))))

NF =  AffineManager(n_flow=n_flow)
NF.create_model(n_pass_through=1,n_cells=1, nn_width=2, NN=[20,20,20], roll_step=1)
#print(v)
v=NF.model(NF.format_input(v))
jac=v[:,-1]

v=v[:,:n_flow]

mult=torch.mul(g(v),jac)

print("Value of the integral after mapping by CL")
print(torch.mean(torch.mul(mult,torch.prod(tanp(w),axis=-1))))
# With Gaussian example:
#dim 2, cells 1: reproduces integral very good
#dim 2, cells 2: slightly more deviation, O(1e-3)
#dim 2, cells 10: O(1e-1)
#dim 4, cells 2: deviation O(1e-1)
#dim 2, cells 4: deviation O(1e-3)
#dim 2, cells 8: deviation O(1e-3)
#dim 8, cells 2: deviation O(10)
#dim 8, cells 4: deviation O(10)
#result stable up to dim 5 -> sampling to sparse in high dimension, no bug


Value of the integral before mapping by CL
tensor(64.0209)
Value of the integral after mapping by CL
tensor(63.7824, grad_fn=<MeanBackward0>)
